In [1]:
%matplotlib widget
from spectraclass.data.base import DataManager
from spectraclass.data.base import ModeDataManager
from spectraclass.data.spatial.tile.manager import TileManager
from spectraclass.data.spatial.modes import AvirisDataManager
from spectraclass.application.controller import app, SpectraclassController
from spectraclass.model.labels import LabelsManager, lm
from spectraclass.learn.manager import ClassificationManager, cm
from spectraclass.learn.base import LearningModel
import os
from typing import List, Union, Tuple, Optional, Dict, Callable

2022-10-10 13:56:31.396844: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
/gpfsm/ccds01/home/appmgr/app/jupyterhub/ilab/kernel/lib/python3.7/site-packages/requests/__init__.py:104: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (5.0.0)/charset_normalizer (2.0.10) doesn't match a supported version!
  RequestsDependencyWarning)


Here we configure paths on the Jupyter server.  If these paths are not specified here then the default values,
    defined in server-side config files, for the project ("demo2") and data mode ("desis"), will be used.  You can
    choose whatever project names you want, they are used to save configurations and results for ongoing investigations.

In [2]:
dm: DataManager = DataManager.initialize( "img_mgr", 'aviris' )
location = "adapt"
if location == "adapt":
    dm.modal.cache_dir = "/explore/nobackup/projects/ilab/cache"
    dm.modal.data_dir = "/css/above/daac.ornl.gov/daacdata/above/ABoVE_Airborne_AVIRIS_NG/data/"
    dm.modal.images_glob =  "ang20170731t222832rfl/ang*_rfl_v2p9/ang*_corr_v2p9_img"
elif location == "desktop":
    dm.modal.cache_dir = "/Volumes/Shared/Cache"
    dm.modal.data_dir = "/Users/tpmaxwel/Development/Data/Aviris"
else: raise Exception( f"Unknown location: {location}")

block_size = 150
method = "aec" # "vae"
model_dims = 32
version = "v2v2"
month = "201908"

dm.modal.ext =  "_img"
dm.use_model_data = True
dm.proc_type = "skl"
dm.modal.images_glob = f"ang{month}*rfl/ang*_rfl_{version}/ang*_corr_{version}_img"
TileManager.block_size = block_size
TileManager.block_index = [1,7]
AvirisDataManager.version = version
dm.modal.valid_aviris_bands = [ [5,193], [214,283], [319,10000] ]
dm.modal.model_dims = model_dims
dm.modal.reduce_method = method
dm.modal.reduce_nepoch = 3
dm.modal.reduce_focus_nepoch = 10
dm.modal.reduce_niter = 1
dm.modal.reduce_focus_ratio = 10.0
dm.modal.reduce_dropout = 0.0
dm.modal.reduce_learning_rate = 1e-4
dm.modal.refresh_model = False
dm.modal.modelkey = f"b{block_size}.{method}"

Opening log file:  '/home/tpmaxwel/.spectraclass/logging/aviris/img_mgr.log'
Using config file: '/panfs/ccds02/home/tpmaxwel/JupyterLinks/spectraclass-dev/defaults/config.py'
Using config file: '/home/tpmaxwel/.spectraclass/config/aviris/img_mgr.py'


/gpfsm/ccds01/home/appmgr/app/jupyterhub/ilab/kernel/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3186: UserWarning: Config option `refresh_data` not recognized by `DataManager`.
  interactivity=interactivity, compiler=compiler, result=result)
/panfs/ccds02/home/tpmaxwel/JupyterLinks/spectraclass-dev/notebooks/spectraclass/model/base.py:20: UserWarning: Config option `reduce_nepoch` not recognized by `AvirisDataManager`.  Did you mean one of: `reduce_method, reduce_nepochs`?
  inst = cls(*args, **kwargs)
/panfs/ccds02/home/tpmaxwel/JupyterLinks/spectraclass-dev/notebooks/spectraclass/model/base.py:20: UserWarning: Config option `reduce_niter` not recognized by `AvirisDataManager`.  Did you mean one of: `reduce_method, reduce_nepochs, reduce_sparsity`?
  inst = cls(*args, **kwargs)
/panfs/ccds02/home/tpmaxwel/JupyterLinks/spectraclass-dev/notebooks/spectraclass/model/base.py:20: UserWarning: Config option `refresh_model` not recognized by `AvirisDataManager`.
  inst = cl

Here we load the project data and define a set of class names (with associated colors) to be used in the classification process.

In [3]:
dm.loadCurrentProject()
classes = [ ('Class-1', "cyan"),
            ('Class-2', "green"),
            ('Class-3', "magenta"),
            ('Class-4', "blue")]

lm().setLabels( classes )
dm.modal.initialize_dimension_reduction()

Completed Reading raster file /css/above/daac.ornl.gov/daacdata/above/ABoVE_Airborne_AVIRIS_NG/data//ang20190802t211646rfl/ang20190802t211646_rfl_v2v2/ang20190802t211646_corr_v2v2_img, dims = ('band', 'y', 'x'), shape = (425, 17869, 709)
Generating metadata: ang20190802t211646rfl/ang20190802t211646_rfl_v2v2/ang20190802t211646_corr_v2v2_img.mdata.txt
 ---> Writing metadata, time = 0.19937886794408163 min
Loading metadata from file: /explore/nobackup/projects/ilab/cache/spectraclass/aviris/img_mgr/ang20190802t211646rfl/ang20190802t211646_rfl_v2v2/ang20190802t211646_corr_v2v2_img.mdata.txt


AssertionError: Project initialization failed- check log file for details

Here we define a custom CNN to use withing the spectraclass framework.

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras import Input
from tensorflow.keras import datasets, layers, models
from spectraclass.learn.models.network import Network, ModelType

class CNN(Network):
    TYPE = ModelType.SPATIAL

    def _build_model(self, **kwargs) -> Tuple[Model,Dict]:
        from spectraclass.learn.models.spatial import SpatialModelWrapper
        nfeatures = kwargs.pop('nfeatures', 64 )
        from spectraclass.model.labels import lm
        input_shape = SpatialModelWrapper.get_input_shape()
        nclasses = lm().nLabels
        ks = kwargs.pop('kernel_size',3)
        model = models.Sequential()
        model.add( Input( shape=input_shape ) )
        model.add( layers.Conv2D( nfeatures, (ks,ks), activation='relu', padding="same" ) )
        model.add( layers.Reshape( SpatialModelWrapper.flatten(input_shape,nfeatures) ) )
        model.add( layers.Dense( nfeatures, activation='relu' ) )
        model.add( layers.Dense( nclasses, activation='softmax' ) )
        return model, kwargs

cm().addNetwork( CNN( 'cnn-1', nfeatures=64, nepochs=200, test_size=0.1 ) )

Here we start up the Spectraclass GUI.

In [ ]:
controller: SpectraclassController = app()
controller.gui()